In [1]:
import requests
import re
import time
import random
import os
import datetime

In [2]:
def update_dmhy_cache(start_page, end_page):
    n = start_page 
    while n <= end_page:
        print("getting page {} ...".format(n))
        try:
            content = requests.get("https://share.dmhy.org/topics/list/page/{}".format(n)).text
            store_cache(content, 'dmhy/cache/{}.txt'.format(n))
            n+=1
        except Exception as e:
            print(datetime.datetime.today(), e)
            time.sleep(8)
        time.sleep(2+random.random())   
        
def store_cache(content, filename):
    table = re.findall(r'<tbody>[\s\S]*</tbody>', content)[0]    
    rows = re.findall(r'<tr[\s\S]*?</tr>',table)                          
    new_items = []
    for i in rows:
        detail = re.findall(r'<td[\s\S]*?</td>',re.sub(r'[\n\t]','',i))  # cols in a row
        release_time = re.findall(r'<span.*?>(.*?)</span>',detail[0])[0]
        release_type = re.sub(r'<.*?>','',detail[1])
        release_title = re.findall(r'<a.*?>(.*?)</a>',detail[2])[-1]
        release_magnet = re.findall( r'href="([^"]*)"',detail[3])[-1]
        release_size = re.sub(r'<.*?>','',detail[4])
        new_items.append(','.join([release_time, release_type, release_title, release_magnet,release_size])+'\n\n')
    with open(filename, 'w', encoding='utf8') as f:
        for i in  reversed(new_items):
            f.write(i)  

In [3]:
def update_dmhy(start_page, end_page):
    for i in reversed(range(start_page, end_page+1)):
        print("updating page {} ...".format(i))
        update_one_page(i)
        
def update_one_page(n):
    with open('dmhy/cache/{}.txt'.format(n), 'r', encoding='utf8') as f:
        records = [i for i in f.readlines() if len(i)>10]
    dict_records = {}
    for i in records:
        year = i[0:4]
        month = i[5:7]
        idx = '{}/{}'.format(year, month)
        filename = 'dmhy/{}-{}.txt'.format(year,month)
        if not idx in dict_records:
            prev_records(year, month, dict_records)
        if not i in dict_records[idx]:
            dict_records[idx].add(i)
            with open(filename,'a+',encoding='utf8') as f:
                f.write(i+'\n')
            
def prev_records(year, month, dict_records):
    filename = 'dmhy/{}-{}.txt'.format(year,month)
    idx = '{}/{}'.format(year, month)
    if not os.path.exists(filename):
        dict_records[idx] = set()
        return 
    with open(filename,'r',encoding='utf8') as f:
        dict_records[idx] = set([i for i in f.readlines() if len(i)>10])

In [4]:
start = 1
end = 9
update_dmhy_cache(start,end)

getting page 1 ...
getting page 2 ...
getting page 3 ...
getting page 4 ...
getting page 5 ...
getting page 6 ...
getting page 7 ...
getting page 8 ...
getting page 9 ...


In [5]:
update_dmhy(start,end)

updating page 9 ...
updating page 8 ...
updating page 7 ...
updating page 6 ...
updating page 5 ...
updating page 4 ...
updating page 3 ...
updating page 2 ...
updating page 1 ...
